# Metabolites

## Description
In this notebook we create a dataframe containing all the available information for the metabolites accounted in our reconstruction.<br>
[A. Generation of Metabolites dataset](#generation) <br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.Retreive info from our reconstruction](#rxns) <br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.Retrieve information from Recon3D, iCHO2291 and iCHO1766](#met) <br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.Add all the metabolites information into our metabolites dataset](#combine) <br><br>
[B. Metabolites Curation](#curation) <br>
&nbsp;&nbsp;&nbsp;&nbsp;[5.Update missing information in metabolites dataset](#missing)

In [ ]:
# Import libraries
import gspread
import pandas as pd
import numpy as np
import requests
import time

import cobra
from cobra.io import read_sbml_model

from tqdm.notebook import tqdm

In [ ]:
# Define functions

def df_to_dict(df, key_col):
    """
    This function takes a pandas dataframe and a key column, and returns a dictionary
    with the key column as the dictionary keys and the rest of the columns as the values.
    """
    # Create an empty dictionary to hold the key-value pairs
    my_dict = {}
    
    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        # Use the value in the key column as the dictionary key
        key_value = row[key_col]
        
        # Use the rest of the columns as the dictionary values
        value_dict = row.drop(key_col).to_dict()
        
        # Add the key-value pair to the dictionary
        my_dict[key_value] = value_dict
    
    return my_dict

<a id='generation'></a>
## A. Generation of Metabolites dataset
We start by creating a list of all the metabolites included in the reactions of our reconstruction (1). Then we create a dataset containing all the metabolites info from Recon3D, iCHO2291 and iCHO1766 models, including supplementary information from Recon 3D (2). Now we can map back this information into the metabolites from our reconstruction and generate an excell file for uploading into Google Sheets (3).

<a id='rxns'></a>
### 1. Retrieve a list of all the metabolites from our reconstruction
The list of all the reactions and the metabolites involved are in the Rxns Sheet in the Google Sheet.

In [ ]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction + Recon3D')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
rxns_sheet = cho_recon.worksheet('Rxns')

In [ ]:
# We can extract the data using the get_all_records method and create a pd DataFrame
df = pd.DataFrame(rxns_sheet.get_all_records())
df

In [ ]:
# Create a cobra model to identify the metabolites involved in our reconstruction
model = cobra.Model("iCHOxxxx")
lr = []

for _, row in df.iterrows():
    r = cobra.Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

In [ ]:
# With the built in function "build_reaction_from_string" we can identify the metabolites
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(df['Reaction Formula'][i])

In [ ]:
# We first create a list of the metabolites and then a pandas df with it
metabolites_list = []
for met in model.metabolites:
    metabolites_list.append(met.id)
    
metabolites = pd.DataFrame(metabolites_list, columns =['BiGG ID'])
metabolites

<a id='met'></a>
### 2. Retrieve information from all the metabolites on Recon3D, iCHO2291 and iCHO1766
We use two datasets for this, first we take information from the Recon3D.xml, iCHO2291.xml and iCHO1766 files from which we get the metabolite ID, Name, Formula and Compartment. We then add the metadata for the available metabolites from Recon3D supplementary files.

In [ ]:
# read the Recon3D model
recon3d_model = read_sbml_model('../Data/GPR_Curation/Recon3D.xml')

In [ ]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment
num_rows = len(recon3d_model.metabolites)
recon3d_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(recon3d_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    recon3d_model_metabolites.iloc[i] = [id_, name, formula, comp]

In [ ]:
recon3d_model_metabolites

In [ ]:
# read the Yeo's model
iCHO2291_model = read_sbml_model('../Data/Reconciliation/models/iCHO2291.xml')

In [ ]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment from Yeo's model
num_rows = len(iCHO2291_model.metabolites)
iCHO2291_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(iCHO2291_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    iCHO2291_model_metabolites.iloc[i] = [id_, name, formula, comp]
    
iCHO2291_model_metabolites['BiGG ID'] = iCHO2291_model_metabolites['BiGG ID'].str.replace("[", "_", regex=False)
iCHO2291_model_metabolites['BiGG ID'] = iCHO2291_model_metabolites['BiGG ID'].str.replace("]", "", regex=False)
iCHO2291_model_metabolites

In [ ]:
# read Hefzi's model
iCHO1766_model = read_sbml_model('../Data/Reconciliation/models/iCHOv1_final.xml')

In [ ]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment from Hefzi's model
num_rows = len(iCHO1766_model.metabolites)
iCHO1766_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(iCHO1766_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    iCHO1766_model_metabolites.iloc[i] = [id_, name, formula, comp]

iCHO1766_model_metabolites

In [ ]:
models_metabolites = pd.concat([recon3d_model_metabolites, iCHO2291_model_metabolites, iCHO1766_model_metabolites])
models_metabolites = models_metabolites.groupby('BiGG ID').first()
models_metabolites = models_metabolites.reset_index(drop = False)
models_metabolites

In [ ]:
#Generation of a dataset containing all the information from Recon3D metabolites Supplementary Data.
recon3d_metabolites_meta = pd.read_excel('../Data/Metabolites/metabolites.recon3d.xlsx', header = 0)
recon3d_metabolites_meta['BiGG ID'] = recon3d_metabolites_meta['BiGG ID'].str.replace("[", "_", regex=False)
recon3d_metabolites_meta['BiGG ID'] = recon3d_metabolites_meta['BiGG ID'].str.replace("]", "", regex=False)
recon3d_metabolites_meta

In [ ]:
# Transformation of the "recon3d_metabolites_meta" into a dict to map it into the "recon3d_model_metabolites"
recon3dmet_dict = df_to_dict(recon3d_metabolites_meta, 'BiGG ID')

In [ ]:
# Mapping into the "recon3d_model_metabolites" dataset
models_metabolites[['KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES', 'INCHI2',
                          'CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = models_metabolites['BiGG ID'].apply(lambda x: pd.Series(recon3dmet_dict.get(x, None), dtype=object))

In [ ]:
models_metabolites

In [ ]:
# Transform the final Recon3D Metabolites dataset into a dictionary to map it into our dataset
final_met_dict = df_to_dict(models_metabolites, 'BiGG ID')

<a id='combine'></a>
### 3. Add all the metabolites information into our metabolites dataset
With the dictionary created in **Step 2** we can use the information to map it in the metabolites dataset created in **Step 1** which contains all the metabolites of our reconstruction.

In [ ]:
metabolites[['Name', 'Formula', 'Compartment', 'KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES',
             'INCHI2','CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = metabolites['BiGG ID'].apply(lambda x: pd.Series(final_met_dict.get(x, None), dtype=object))

In [ ]:
# Update the Compartment column in the final dataset
for i,row in metabolites.iterrows():
    if row['Compartment'] == 'c':
        metabolites.loc[i, 'Compartment'] = 'c - cytosol'
    if row['Compartment'] == 'l':
        metabolites.loc[i, 'Compartment'] = 'l - lysosome'
    if row['Compartment'] == 'm':
        metabolites.loc[i, 'Compartment'] = 'm - mitochondria'
    if row['Compartment'] == 'r':
        metabolites.loc[i, 'Compartment'] = 'r - endoplasmic reticulum'
    if row['Compartment'] == 'e':
        metabolites.loc[i, 'Compartment'] = 'e - extracellular space'
    if row['Compartment'] == 'x':
        metabolites.loc[i, 'Compartment'] = 'x - peroxisome/glyoxysome'
    if row['Compartment'] == 'n':
        metabolites.loc[i, 'Compartment'] = 'n - nucleus'
    if row['Compartment'] == 'g':
        metabolites.loc[i, 'Compartment'] = 'g - golgi apparatus'
    if row['Compartment'] == 'im':
        metabolites.loc[i, 'Compartment'] = 'im - intermembrane space of mitochondria'

In [ ]:
metabolites.to_excel('../Data/Metabolites/metabolites.xlsx')

### 4. Unique metabolite identification

In [ ]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction + Recon3D')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create a pd DataFrame
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())
metabolites

In [ ]:
print("Duplicated rxns by BiGG ID = ", len(metabolites['BiGG ID']) - len(metabolites['BiGG ID'].unique()))
print("Duplicated rxns by Name = ", len(metabolites['Name']) - len(metabolites['Name'].unique()))
print("Duplicated rxns by Formula = ", len(metabolites['Formula']) - len(metabolites['Formula'].unique()))
print("Duplicated rxns by KEGG = ", len(metabolites['KEGG']) - len(metabolites['KEGG'].unique()))

In [ ]:
# Group the metabolites by formula and create an excel file to curate them manually.
grouped_by_formula = metabolites.groupby('Formula', group_keys=True).apply(lambda x:x)
grouped_by_formula.to_excel('../Data/Metabolites/grouped_by_formula_recon3d.xlsx')

<a id='curation'></a>
## B. Metabolites Curation
In this second part of the notebook we curate missing information in the metabolites dataset generated above. Since many metabolites have been manually curated in the "Metabolites" google sheet file, we generate a new dataframe using the gspread library to obtain the metabolites dataset with all the changes

<a id='missing'></a>
### 5. Update missing information in metabolites dataset

In [ ]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction + Recon3D')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')
rxns_sheet = cho_recon.worksheet('Rxns')
attributes_sheet = cho_recon.worksheet('Attributes')

# We can extract the data using the get_all_records method and create a pd DataFrame
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())
rxns = pd.DataFrame(rxns_sheet.get_all_records())
attributes = pd.DataFrame(attributes_sheet.get_all_records())

In [ ]:
# Get BiGG descriptive names
import requests
from bs4 import BeautifulSoup
import time

# Unknown Mets: metabolites without names
unkown_mets = metabolites[metabolites['Name'] == '']

Descriptive_Names = [''] * len(unkown_mets)
Formulae = [''] * len(Descriptive_Names)
Changed = [True] * len(Descriptive_Names)

for Met_Counter, metID in enumerate(tqdm(unkown_mets['BiGG ID'].iloc[:])):
    print(Met_Counter)
    input_str = metID[:-2]
    response = requests.get(f"http://bigg.ucsd.edu/universal/metabolites/{input_str}")
    time.sleep(1)
    # Check if the request was successful
    if response.status_code != 200:
        D_Name = "BiGG ID not found in BiGG"
        Formulae_B = "BiGG ID not found in BiGG"
        Changed[Met_Counter] = False       
    else:    
        soup = BeautifulSoup(response.content, 'html.parser')
        N_Header = soup.find('h4', string='Descriptive name:')
        D_Name = N_Header.find_next_sibling('p').text
        N_Formulae = soup.find('h4', string='Formulae in BiGG models: ')
        Formulae_B = N_Formulae.find_next_sibling('p').text    
        if D_Name is None:
            D_Name = "Name not found in BiGG"            
        elif Formulae_B is None:
            Formulae_B = "Formula not found in BiGG"                
    Descriptive_Names[Met_Counter] = D_Name
    Formulae[Met_Counter] = Formulae_B

### Update empty metabolites

In [ ]:
for Met_Counter, metID in enumerate(unkown_mets['BiGG ID']):
    print('before',unkown_mets['BiGG ID'].iloc[Met_Counter])
    print('before',unkown_mets['Formula'].iloc[Met_Counter])
    print('before',unkown_mets['Name'].iloc[Met_Counter])
    if unkown_mets['Formula'].iloc[Met_Counter] == '':
        unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]  
    unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
    print('..............................................')
    print('after',unkown_mets['BiGG ID'].iloc[Met_Counter])
    print('after',unkown_mets['Formula'].iloc[Met_Counter])
    print('after',unkown_mets['Name'].iloc[Met_Counter])
    print('..............................................')
    print('..............................................')
    print('..............................................')

In [ ]:
metabolites.update(unkown_mets)

# Manual Curation
for bigg_id in metabolites['BiGG ID']:
    # xtra = Xanthurenic acid; C10H6NO4
    # http://bigg.ucsd.edu/models/iCHOv1/reactions/r0647
    if 'xtra' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Xanthurenic acid'
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Formula'] = 'C10H6NO4'
    # chedxch = Bilirubin-monoglucuronoside; C39H42N4O122-
    # Reactions name = 'ATP-binding Cassette (ABC) TCDB:3.A.1.208.2' --> https://metabolicatlas.org/identifier/TCDB/3.A.1.208.2
    elif 'chedxch' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Bilirubin-monoglucuronoside'
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Formula'] = 'C39H42N4O122-'
    # chatGTP
    elif '3hoc246_6Z_9Z_12Z_15Z_18Z_21Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a 24-carbon fatty acid with six double bonds, with the location of the double bonds specified by the numbers and Zs'
    # chatGTP
    elif 'c247_2Z_6Z_9Z_12Z_15Z_18Z_21Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a modified version of the same 24-carbon fatty acid, with a hydroxyl group added at the third carbon position'
    # chatGTP
    elif '3hoc143_5Z_8Z_11Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a 14-carbon fatty acid with three double bonds, with the location of the double bonds specified by the numbers and Zs.'
    # chatGTP
    elif '3oc143_5Z_8Z_11Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a modified version of the same 14-carbon fatty acid, with the hydroxyl group removed and one of the double bonds converted to a keto group'
    # chatGTP
    elif 'acgalgalacglcgalgluside' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Complex glycosphingolipid that contains multiple sugar residues'

    # 12e8hdx WTF?
    # hdxur Dead End

metabolites.to_excel('../Data/Metabolites/metabolites_final.xlsx')

### 6. Identification of duplicated metabolites
The idea here is to add the metabolites from the reactions added from Recon 3D, at the same time that we do not over write data from our own Metabolites dataset.

In [ ]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction + Recon3D_v2')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
met_sheet = cho_recon.worksheet('Metabolites')
rxns_sheet = cho_recon.worksheet('Rxns')
attributes_sheet = cho_recon.worksheet('Attributes')

# We can extract the data using the get_all_records method and create a pd DataFrame
met = pd.DataFrame(met_sheet.get_all_records())
rxns = pd.DataFrame(rxns_sheet.get_all_records())
attributes = pd.DataFrame(attributes_sheet.get_all_records())

In [ ]:
# Convert metabolites names to lower case and remove the compartment
met['Name'] = met['Name'].str.lower()
met_copy = met.copy()
met_copy['BiGG ID'] = met_copy['BiGG ID'].str[:-2]
met_copy

In [ ]:
# Generate a list with duplicated metabolites

grouped = met_copy.groupby(['Name', 'Formula'])

# Initialize an empty dictionary to store the results
duplicated_metabolites = []

# Iterate over the grouped DataFrame
for (Name, Formula), group in grouped:
    # Check if the group has more than one element (i.e., duplicate) and filter out those metabolites whose names are unknown
    if group['BiGG ID'].nunique() > 1 and Name != 'bigg id not found in bigg':
        unique_ids = group['BiGG ID'].unique()
        duplicated_metabolites.append((Name, Formula, unique_ids))

        


In [ ]:
len(duplicated_metabolites)

In [ ]:
# Generate empty dict to store the existence of each duplicated metabolite in BiGG
duplicated_dict = {}


for metabolite in tqdm(duplicated_metabolites):
    duplicated_dict[metabolite[0]] = {}
    for big_id in metabolite[2]:
        time.sleep(1)
        # Check if the metabolite is in BiGG "OK" or not "NO"
        response = requests.get(f"http://bigg.ucsd.edu/universal/metabolites/{big_id}")
        if response.status_code == 200:
            duplicated_dict[metabolite[0]][big_id] = 'OK'
        else:
            duplicated_dict[metabolite[0]][big_id] ='NO'
        


In [ ]:
duplicated_dict

In [ ]:
# Create a dictionary to store the 'OK' subkey for each key in duplicated_dict
ok_dict = {}

# Iterate over keys in duplicated_dict
for key in duplicated_dict:
    # Create an empty list to store 'NO' subkeys for this key
    no_list = []
    # Iterate over subkeys and values in sub-dictionary
    for subkey, value in duplicated_dict[key].items():
        # If the value is 'OK', save the subkey to a variable
        if value == 'OK':
            ok_dict[key] = subkey
        # If the value is 'NO', add the subkey to the list
        elif value == 'NO':
            no_list.append(subkey)
    # Replace all 'NO' subkeys with the 'OK' subkey for this key
    if key in ok_dict:
        ok_subkey = ok_dict[key]
        for no_subkey in no_list:
            met['BiGG ID'] = met['BiGG ID'].str.replace(no_subkey, ok_subkey)
            rxns['Reaction Formula'] = rxns['Reaction Formula'].str.replace(no_subkey, ok_subkey)
            attributes['Reaction Formula'] = attributes['Reaction Formula'].str.replace(no_subkey, ok_subkey)
    # Reset the 'ok_subkey' and 'no_subkey' variables at the end of each iteration over keys
    ok_dict[key] = None

In [ ]:
met = met.groupby('BiGG ID').first()
met

In [ ]:
met.to_excel('met.xlsx')
rxns.to_excel('rxns.xlsx')
attributes.to_excel('attribute.xlsx')

In [2]:
from google_sheet import GoogleSheet

KEY_FILE_PATH = '/Users/pablodigiusto/Documents/GitHub/Whole-Cell-Network-Reconstruction-for-CHO-cells/Notebooks/credentials.json'
SPREADSHEET_ID = '1ue3VOG07G1IGaQ6xBkEcu7Q1rB8mbncO1pP_T9Eb7gw'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_name = 'Rxns'
dataframe = sheet.read_google_sheet(sheet_name)
print("Dataframe from Google Sheet:\n", dataframe)

# Make some changes to the dataframe
# ...

# Update the Google Sheet with the modified DataFrame
sheet.update_google_sheet(sheet_name, dataframe)
print("Google Sheet updated.")

Dataframe from Google Sheet:
      Curated      Reaction                               Reaction Name  \
0         PD  10FTHF5GLUtl      5-glutamyl-10fthf transport, lysosomal   
1         PD  10FTHF5GLUtm  5-glutamyl-10fthf transport, mitochondrial   
2         PD  10FTHF6GLUtl      6-glutamyl-10fthf transport, lysosomal   
3         PD  10FTHF6GLUtm  6-glutamyl-10fthf transport, mitochondrial   
4         PD  10FTHF7GLUtl      7-glutamyl-10fthf transport, lysosomal   
...      ...           ...                                         ...   
8195                 r2534      major facilitator(mfs) tcdb:2.a.1.44.1   
8196                 r2535      major facilitator(mfs) tcdb:2.a.1.44.1   
8197                 r2537                          utilized transport   
8198                 r2538                          utilized transport   
8199                 r2539               postulated transport reaction   

                                       Reaction Formula  \
0                     